In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!python --version

Python 3.10.12


In [1]:
!pip install -q langchain
!pip install -qU langchain-openai
!pip install -q langchain-community
!pip install -q langchain_chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import getpass
import os

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_TRACING_V2"] = "false"

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

··········


In [3]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [21]:
from bs4 import BeautifulSoup
import re
import json

with open('/content/drive/MyDrive/makarem.xml', 'r', encoding="utf8") as f:
  content = f.read()

soup= BeautifulSoup(content, 'xml')

dict_data_list = []

verses = soup.find_all("s")
for verse in verses:
  context = re.sub("[\(\[].*?[\)\]]", "", verse.contents[0])
  dict_data_list.append({'content': context, 'id':verse.get('id')})

data = {}
data['data'] = dict_data_list

with open('/content/drive/MyDrive/makarem.json', 'w') as f:
  json.dump(data, f, ensure_ascii = False)


In [4]:
!pip install jq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.6/657.6 kB 10.3 MB/s eta 0:00:00


In [7]:
from langchain_community.document_loaders import JSONLoader
from pprint import pprint


loader = JSONLoader(
    file_path='/content/drive/MyDrive/makarem.json',
    jq_schema='.data[].content',
    text_content=False,)

data = loader.load()

# pprint(data)

In [8]:
docs = data
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# splits = text_splitter.split_documents(docs)
# vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# # Retrieve and generate using the relevant snippets of the blog.
# retriever = vectorstore.as_retriever()

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

6237

In [9]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

OPENAI_API_KEY = ''
# vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())
db_chroma = Chroma.from_documents(documents=all_splits,
                                  embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY))


In [34]:
query = 'what are the top risks mentioned in the document?'
# retrieve context - top 5 most relevant (closests) chunks to the query vector
# (by default Langchain is using cosine distance metric)
docs_chroma = db_chroma.similarity_search_with_score(query, k=5)


In [16]:
from langchain.prompts import ChatPromptTemplate

query = 'برادران حضرت یوسف او را در چه مکانی افکندند?'
# retrieve context - top 5 most relevant (closests) chunks to the query vector
# (by default Langchain is using cosine distance metric)
docs_chroma = db_chroma.similarity_search_with_score(query, k=1)
# generate an answer based on given user query and retrieved context information
context_text = "\n\n".join([doc.page_content for doc, _score in docs_chroma])
# you can use a prompt template
PROMPT_TEMPLATE = """
فقط بر اساس متن زیر به پرسش پاسخ دهید:
{context}
پرسش را با توجه به متن بالا پاسخ بده: {question}.
یک پاسخ کوتاه و بدون جمله کامل بده.
برای پاسخ دلیل و مدرک ارائه نده.
از توضیحات اضافی خودداری کن.
"""
# load retrieved context and user query in the prompt template
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query)
# call LLM model to generate the answer based on the given context and query
model = ChatOpenAI()
response_text = model.predict(prompt)

In [17]:
response_text

'چاه'

# run for Quran Dataset

In [18]:
import json

with open('/content/drive/MyDrive/train_v9.json', 'r', encoding = 'utf-8-sig') as f:
  train_list = json.load(f)

with open('/content/drive/MyDrive/dev_v9.json', 'r', encoding = 'utf-8-sig') as f:
  dev_list = json.load(f)

total_data = train_list + dev_list

In [19]:
non_referral = []
referral = []

str1 = 'سوره'
str2 = 'نام ببرید'
str3 = 'یکی از انواع'

for qa in total_data:
  question = qa['question']
  if str1 not in question and str2 not in question and str3 not in question:
    non_referral.append(qa)
  else:
    referral.append(qa)

# test set

In [21]:
with open('/content/drive/MyDrive/dev_10_percent.json', 'r', encoding = 'utf-8-sig') as f:
  test_data = json.load(f)

In [25]:
new_list = []

for qa in test_data:
  query = qa['question']
  docs_chroma = db_chroma.similarity_search_with_score(query, k=1)
  context_text = "\n\n".join([doc.page_content for doc, _score in docs_chroma])
  # you can use a prompt template
  PROMPT_TEMPLATE = """
  فقط بر اساس متن زیر به پرسش پاسخ دهید:
  {context}
  پرسش را با توجه به متن بالا پاسخ بده: {question}.
  یک پاسخ کوتاه و بدون جمله کامل بده.
  برای پاسخ دلیل و مدرک ارائه نده.
  از توضیحات اضافی خودداری کن.
  """
  # load retrieved context and user query in the prompt template
  prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
  prompt = prompt_template.format(context=context_text, question=query)
  # call LLM model to generate the answer based on the given context and query
  response_text = model.predict(prompt)
  qa['gpt'] = response_text
  new_list.append(qa)

with open('/content/drive/MyDrive/RAG_part1.json', 'w', encoding = 'utf-8-sig') as f:
  json.dump(new_list, f, ensure_ascii= False)

## evaluate

In [26]:
!pip install -q evaluate
!pip install -q parsivar

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 35.4 MB/s eta 0:00:00


In [29]:
from evaluate import load
import json
from parsivar import Normalizer

my_normalizer = Normalizer()

with open('/content/drive/MyDrive/RAG_part1.json', 'r', encoding = 'utf-8-sig') as f:
  data_list = json.load(f)

predictions = []
references = []

for data in data_list:
  if data['answers'][0]['text'] in data['gpt'] or data['gpt'] in data['answers'][0]['text']:
    # predictions.append(data['answers'][0]['text'])
    # references.append(data['answers'][0]['text'])
    predictions.append(my_normalizer.normalize(data['answers'][0]['text']))
    references.append(my_normalizer.normalize(data['answers'][0]['text']))
  else:
    # predictions.append(data['gpt'].strip())
    # references.append(data['answers'][0]['text'])
    predictions.append(my_normalizer.normalize(data['gpt'].strip()))
    references.append(my_normalizer.normalize(data['answers'][0]['text']))

exact_match_metric = load("exact_match")
results = exact_match_metric.compute(predictions=predictions, references=references)

In [30]:
results

{'exact_match': 0.15072083879423329}

In [31]:
from sklearn.metrics import f1_score, precision_score, recall_score

# f1 = f1_score(ground_truth, predicted_output, average='micro')
f1 = f1_score(references, predictions, average='weighted')

f1

0.17303091378336155